<a href="https://colab.research.google.com/github/ft-Azad/Language-Modeling/blob/main/LanguageModel(AWD_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Import Libs

In [ ]:
!pip install -q torchmetrics
!pip install -q torchdata==0.6.1
!pip install -q 'portalocker>=2.0.0'
!pip install -q torchtext==0.15.1
!pip install -q comet_ml

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torchtext
from torchtext.datasets import WikiText2
from torchdata.datapipes.iter import IterableWrapper, Mapper
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split

from torch import optim
from torch.nn import functional as F

import os
import tqdm
import torchmetrics as tm

from collections import Counter

from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [2]:
for lib in [np, torch, torchtext, tqdm]:
  print(lib.__name__, '-->', lib.__version__)

numpy --> 1.25.2
torch --> 2.0.0+cu117
torchtext --> 0.15.1+cpu
tqdm --> 4.66.4


# Utils

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def num_trainable_params(model):
  nums = sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6
  return nums

In [5]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
      torch.cuda.manual_seed(seed)
      # torch.cuda.manual_seed_all(seed)

      # torch.backends.cudnn.deterministic = True
      # torch.backends.cudnn.benchmark = False


# Arguments

In [6]:
seed = 8

batch_size = 80
seq_len = 70

embedding_dim = 400

num_layers = 3
hidden_dim = 1150
dropout_embd = 0.5
dropout_rnn = 0


lr = 3
wd = 1e-6
momentum = 0.9

clip = 0.25

wandb_enable = True

#Dataset

## Load Dataset and Build Vocab

- Loading Iterable Data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!unzip -q "/content/drive/MyDrive/LanguageModel/Data/wikitext-2-v1.zip" -d '/content/'

In [14]:
with open('/content/wikitext-2/wiki.train.tokens', 'r') as file:
    train_iter = file.read().splitlines()

with open('/content/wikitext-2/wiki.test.tokens', 'r') as file:
    test_iter = file.read().splitlines()

with open('/content/wikitext-2/wiki.valid.tokens', 'r') as file:
    valid_iter = file.read().splitlines()

train_iter = IterableWrapper(train_iter)
test_iter = IterableWrapper(test_iter)
valid_iter = IterableWrapper(valid_iter)

- Build Vocabulary

In [15]:
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
torch.save(vocab, 'vocab.pt')

## Dataset Prepreation

- Creat Target Array

In [16]:
def data_process(raw_text_iter, seq_len):
  data = torch.cat([torch.LongTensor(vocab(tokenizer(line))) for line in raw_text_iter])

  M = len(data) // seq_len

  r = len(data) % seq_len
  data = torch.cat((data, torch.LongTensor([0]))) if r==0 else data

  inputs = data[:M*seq_len]
  targets = data[1:M*seq_len+1]

  inputs = inputs.reshape(-1, seq_len)
  targets = targets.reshape(-1, seq_len)

  return inputs, targets

In [17]:
X_train, y_train = data_process(train_iter, seq_len)
X_valid, y_valid = data_process(valid_iter, seq_len)
X_test, y_test = data_process(test_iter, seq_len)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

(torch.Size([29285, 70]),
 torch.Size([29285, 70]),
 torch.Size([3063, 70]),
 torch.Size([3063, 70]),
 torch.Size([3455, 70]),
 torch.Size([3455, 70]))

- Custom Dataset Class Definition

In [18]:
class LanguageModelDataset(Dataset):

  def __init__(self, inputs, targets):
    self.inputs = inputs
    self.targets = targets

  def __len__(self):
    return self.inputs.shape[0]

  def __getitem__(self, idx):
    return self.inputs[idx], self.targets[idx]

In [19]:
train_set = LanguageModelDataset(X_train, y_train)
valid_set = LanguageModelDataset(X_valid, y_valid)
test_set = LanguageModelDataset(X_test, y_test)

In [20]:
train_set[0]

(tensor([    9,  3849,  3869,   881,     9, 20000,    83,  3849,    88,     0,
          3869,    21,   780, 28780,     2,  6182,     3,  3849,     4,     1,
          5023,    88,    20,     2,  1837,  1018,     7,    14,  3849,  3869,
           881,   629,   976,     2,    23,     8,  5790,   299,    12,   575,
           232,    67,   452,    19, 13722,     5,   757,     3,  2500,    17,
             1,  1767,  5637,     3,   155,     6,   246,   354,     6,   976,
             2,    24,    23,     1,   237,    67,     6,     1,  3849,    93]),
 tensor([ 3849,  3869,   881,     9, 20000,    83,  3849,    88,     0,  3869,
            21,   780, 28780,     2,  6182,     3,  3849,     4,     1,  5023,
            88,    20,     2,  1837,  1018,     7,    14,  3849,  3869,   881,
           629,   976,     2,    23,     8,  5790,   299,    12,   575,   232,
            67,   452,    19, 13722,     5,   757,     3,  2500,    17,     1,
          1767,  5637,     3,   155,     6,   246,

- Data Loader

In [21]:
set_seed(seed)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [22]:
x_batch, y_batch = next(iter(train_loader))
x_batch.shape, y_batch.shape

(torch.Size([80, 70]), torch.Size([80, 70]))

In [23]:
set_seed(seed)

for inputs, targets in train_loader:
  print(inputs[0, 0], targets[0, 0])
  break

tensor(1985) tensor(13)


# Model

In [45]:
class LanguageModel(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers,
                dropout_embd=0.5, dropout_rnn=0.5):
    super().__init__()
    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.embedding.weight.data.uniform_(-0.1, 0.1)
    self.dropout = nn.Dropout(p=dropout_embd)

    self.lstms = []
    self.lstms.append(nn.LSTM(embedding_dim, hidden_dim, num_layers=1, dropout=0, batch_first=True))
    self.lstms.append(nn.LSTM(hidden_dim, hidden_dim, num_layers=1, dropout=0, batch_first=True))
    self.lstms.append(nn.LSTM(hidden_dim, embedding_dim, num_layers=1, dropout=0, batch_first=True))
    self.lstms = nn.ModuleList(self.lstms)

    self.fc = nn.Linear(embedding_dim, vocab_size)

    self.fc.weight = self.embedding.weight

  def forward(self, src):
    embedding = self.dropout(self.embedding(src))

    # new_hiddens = []
    for l, lstm in enumerate(self.lstms):
      embedding, _ = lstm(embedding)

    prediction = self.fc(embedding)
    return prediction

In [25]:
set_seed(seed)

model = LanguageModel(vocab_size=len(vocab),
                      embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim,
                      num_layers=num_layers,
                      dropout_embd=dropout_embd,
                      dropout_rnn=dropout_rnn)
model


LanguageModel(
  (embedding): Embedding(28782, 400)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstms): ModuleList(
    (0): LSTM(400, 1150, batch_first=True)
    (1): LSTM(1150, 1150, batch_first=True)
    (2): LSTM(1150, 400, batch_first=True)
  )
  (fc): Linear(in_features=400, out_features=28782, bias=True)
)

# Configuration

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
loss_fn = nn.CrossEntropyLoss()

metric = tm.text.Perplexity().to(device)

In [28]:
key_file = "/content/drive/MyDrive/LanguageModel/key.txt"

if os.path.exists(key_file):
    with open(key_file) as f:
        key = f.readline().strip()
else:
    print("Key file does not exist. Please create the key file with your wandb API key.")

In [49]:
wandb_arg_name = input('Please input the WandB argument (run) name:')

Please input the WandB argument (run) name:Weight-Tying


In [30]:
# comet_ml.init(project_name="LM_AWD-LSTM")

In [50]:
experiment = Experiment(
  api_key=key,
  project_name="LM_AWD-LSTM",
)
experiment.set_name(wandb_arg_name)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/ft-azad/lm-awd-lstm/10b4fa44a0994699add9a22de50af0ba



# Train

## Train and Evaluate Functions

In [32]:
# Train Functions
def train_one_epoch(model, train_loader, loss_fn, optimizer, metric, epoch=None):
  model.train()
  loss_train = AverageMeter()
  metric.reset()

  with tqdm.tqdm(train_loader, unit='batch') as tepoch:
    for inputs, targets in tepoch:
      if epoch:
        tepoch.set_description(f'Epoch {epoch}')

      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)

      loss = loss_fn(outputs.reshape(-1, outputs.shape[-1]), targets.flatten())

      loss.backward()

      nn.utils.clip_grad.clip_grad_norm_(model.parameters(), max_norm=clip)

      optimizer.step()
      optimizer.zero_grad()

      loss_train.update(loss.item(), n=len(targets))
      metric.update(outputs, targets)

      tepoch.set_postfix(loss=loss_train.avg, metric=metric.compute().item())

  return model, loss_train.avg, metric.compute().item()

In [33]:
# Evaluate Function
def evaluate(model, test_loader, loss_fn, metric):
  model.eval()
  loss_eval = AverageMeter()
  metric.reset()

  with torch.inference_mode():
    for inputs, targets in test_loader:
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)

      loss = loss_fn(outputs.reshape(-1, outputs.shape[-1]), targets.flatten())
      loss_eval.update(loss.item(), n=len(targets))

      metric(outputs, targets)

  return loss_eval.avg, metric.compute().item()

## Train Process and Tunning

### Finding Hyper-parameters

- Calculating the loss for untrained model using a few batches

In [ ]:
for iter_num in range(5):
  model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim, num_layers=num_layers,
                      dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)

  inputs, targets = next(iter(train_loader))
  inputs = inputs.to(device)
  targets = targets.to(device)

  with torch.no_grad():
    outputs = model(inputs)
    loss = loss_fn(outputs.reshape(-1, outputs.shape[-1]), targets.flatten())

  print(loss)

tensor(10.2711, device='cuda:0')
tensor(10.2717, device='cuda:0')
tensor(10.2653, device='cuda:0')
tensor(10.2671, device='cuda:0')
tensor(10.2690, device='cuda:0')


- Train and try to overfit the model on a small subset of the dataset.

In [ ]:
model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim, num_layers=num_layers,
                      dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.9, momentum=0.9)

In [ ]:
mini_train_size = 1000
_, mini_train_dataset = random_split(train_set, (len(train_set)-mini_train_size, mini_train_size))
mini_train_loader = DataLoader(mini_train_dataset, 20)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
  model, _, _ = train_one_epoch(model, mini_train_loader, loss_fn, optimizer, metric, epoch)

Epoch 56:  62%|██████▏   | 31/50 [00:01<00:00, 24.28batch/s, loss=1.98, metric=7.23]


KeyboardInterrupt: 

-  Train the model for a limited number of epochs, experimenting with various learning rates to find best value

In [ ]:
num_epochs = 2

for lr in [12, 8, 4, 2, 0.9]:
  print(f'LR={lr}')

  model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                        hidden_dim=hidden_dim, num_layers=num_layers,
                        dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)
  # model = torch.load('model.pt')

  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=momentum)

  for epoch in range(num_epochs):
    model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, metric, epoch)

  print()

LR=12


Epoch 1:  90%|████████▉ | 2624/2929 [01:47<00:12, 24.32batch/s, loss=5.65, metric=284]


KeyboardInterrupt: 

- Creat a small grid search to find exact value of lr and weight decay

In [ ]:
num_epochs = 1

for lr in [7, 8, 14, 13, 12, 11, 10, 9]:
  for wd in [1.2e-6]:
    print(f'LR={lr}, WD={wd}')

    model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim, num_layers=num_layers,
                      dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)

    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)

    for epoch in range(num_epochs):
      model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, metric, epoch)

    print()

- Train model for more epochs using the best hyperparameters

In [ ]:
model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim, num_layers=num_layers,
                      dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)
model

LanguageModel(
  (embedding): Embedding(28782, 300)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(300, 512, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=28782, bias=True)
)

In [ ]:
model = torch.load('/content/model-ppl_133.pt')

FileNotFoundError: [Errno 2] No such file or directory: '/content/model-ppl_133.pt'

In [ ]:
lr = 8
wd = 1e-6
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 8
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 1e-06
)

In [ ]:
loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0

In [ ]:
num_epochs = 30

for epoch in range(1, num_epochs+1):
  # WandB
  # run = wandb.init(
  #       project="language-modeling-lstms",
  #       config={
  #           "learning_rate": lr,
  #           "epochs": num_epochs,
  #       })

  # Train
  model, loss_train, metric_train = train_one_epoch(model,
                                                    train_loader,
                                                    loss_fn,
                                                    optimizer,
                                                    metric,
                                                    epoch)
  # Validation
  loss_valid, metric_valid = evaluate(model,
                                      valid_loader,
                                      loss_fn,
                                      metric)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  metric_train_hist.append(metric_train)
  metric_valid_hist.append(metric_valid)

  if loss_valid < best_loss_valid:
    torch.save(model, f'model.pt')
    best_loss_valid = loss_valid
    print('Model Saved!')

  print(f'Valid: Loss = {loss_valid:.4}, Metric = {metric_valid:.4}')
  print()

  epoch_counter += 1

Epoch 1: 100%|██████████| 2929/2929 [01:59<00:00, 24.46batch/s, loss=5.88, metric=360]


Model Saved!
Valid: Loss = 6.046, Metric = 422.9



Epoch 2: 100%|██████████| 2929/2929 [01:59<00:00, 24.49batch/s, loss=5.24, metric=188]


Model Saved!
Valid: Loss = 6.027, Metric = 414.9



Epoch 3: 100%|██████████| 2929/2929 [01:59<00:00, 24.48batch/s, loss=4.96, metric=143]


...

Valid: Loss = 6.064, Metric = 430.6



Epoch 5: 100%|██████████| 2929/2929 [01:59<00:00, 24.45batch/s, loss=4.65, metric=105]


Valid: Loss = 6.068, Metric = 432.1



Epoch 6:  40%|████      | 1177/2929 [00:48<01:11, 24.51batch/s, loss=4.5, metric=90]


KeyboardInterrupt: 

### Main Train Loop

In [51]:
torch.cuda.empty_cache()

In [52]:
set_seed(seed)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [53]:
set_seed(seed)

model = LanguageModel(len(vocab), embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim, num_layers=num_layers,
                      dropout_embd=dropout_embd, dropout_rnn=dropout_rnn).to(device)

# model = LanguageModel(vocab_size=len(vocab), embedding_dim=embedding_dim,
#                       hidden_dim=hidden_dim, num_layers=num_layers,
#                       dropoute=dropoute, dropouti=dropouti,
#                       dropouth=dropouth, dropouto=dropouto,
#                       weight_drop=weight_drop, pretrained=pretrained).to(device)
model

LanguageModel(
  (embedding): Embedding(28782, 400)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstms): ModuleList(
    (0): LSTM(400, 1150, batch_first=True)
    (1): LSTM(1150, 1150, batch_first=True)
    (2): LSTM(1150, 400, batch_first=True)
  )
  (fc): Linear(in_features=400, out_features=28782, bias=True)
)

In [58]:
model = torch.load('model.pt')
# epoch_counter = best_epoch

In [ ]:
shutil.copyfile("model-ppl130.8.pt", "/content/drive/MyDrive/LanguageModel/model-ppl130.8.pt")

'/content/drive/MyDrive/LanguageModel/model-ppl130.8.pt'

In [59]:
set_seed(seed)

lr = 1.25
wd = 1e-6
# momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
# optimizer = optim.SGD([{'params': model.embedding.parameters(), 'lr': 0.1*lr},
#                        {'params': model.lstms.parameters(), 'lr': lr}],
#                       weight_decay=wd, momentum=momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 1.25
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 1e-06
)

In [55]:
if wandb_enable:

      config={
          'lr': lr,
          'momentum': momentum,
          'batch_size': batch_size,
          'seq_len': seq_len,
          'hidden_dim': hidden_dim,
          'embedding_dim': embedding_dim,
          'num_layers': num_layers,
          'dropout_embd': dropout_embd,
          'dropout_rnn': dropout_rnn,
          # 'dropout_embed': dropoute,
          # 'dropout_in_lstm': dropouti,
          # 'dropout_h_lstm': dropouth,
          # 'dropout_out_lstm': dropouto,
          'clip': clip,
      }

      experiment.log_parameters(config)

In [56]:
loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 1

In [60]:
set_seed(seed)
num_epochs = 4

for epoch in range(1, num_epochs+1):
  # Train
  model, loss_train, metric_train = train_one_epoch(model,
                                                    train_loader,
                                                    loss_fn,
                                                    optimizer,
                                                    metric,
                                                    epoch)
  # Validation
  loss_valid, metric_valid = evaluate(model,
                                      valid_loader,
                                      loss_fn,
                                      metric)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  metric_train_hist.append(metric_train)
  metric_valid_hist.append(metric_valid)

  if loss_valid < best_loss_valid:
    torch.save(model, f'model.pt')
    best_loss_valid = loss_valid
    best_epoch = epoch_counter
    print('Model Saved!')

  print(f'Valid: Loss = {loss_valid:.4}, Metric = {metric_valid:.4}')
  print()

  if wandb_enable:
    experiment.log_metrics({"metric_train": metric_train, "loss_train": loss_train,
                "metric_valid": metric_valid, "loss_valid": loss_valid}, epoch=epoch_counter)

  epoch_counter += 1

Epoch 1: 100%|██████████| 367/367 [02:34<00:00,  2.37batch/s, loss=4.04, metric=56.7]


Model Saved!
Valid: Loss = 4.73, Metric = 113.3



Epoch 2: 100%|██████████| 367/367 [02:34<00:00,  2.37batch/s, loss=3.93, metric=50.9]


Valid: Loss = 4.74, Metric = 114.4



Epoch 3: 100%|██████████| 367/367 [02:35<00:00,  2.37batch/s, loss=3.86, metric=47.5]


Valid: Loss = 4.754, Metric = 116.0



Epoch 4: 100%|██████████| 367/367 [02:34<00:00,  2.37batch/s, loss=3.8, metric=44.5]


Valid: Loss = 4.769, Metric = 117.8



In [61]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : Weight-Tying
COMET INFO:     url                   : https://www.comet.com/ft-azad/lm-awd-lstm/10b4fa44a0994699add9a22de50af0ba
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss_train [14]   : (3.7961325479363834, 6.5045131098314295)
COMET INFO:     loss_valid [14]   : (4.729722857747714, 5.689316993984879)
COMET INFO:     metric_train [14] : (44.528602600097656, 668.1500854492188)
COMET INFO:     metric_valid [14] : (113.26423645019531, 295.691650390625)
COMET INFO:   Others:
COMET INFO:     Name : Weight-Tying
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 80
COMET INFO:     clip          : 0.25
COMET INFO:     dropout_embd 

# Test

In [ ]:
model_path = 'model.pt'
model = torch.load(model_path)
model.eval()

In [ ]:
loss_valid, metric_valid = evaluate(model, valid_loader, loss_fn, metric)
metric_valid

In [ ]:
loss_test, metric_test = evaluate(model, test_loader, loss_fn, metric)
metric_test

# Generate

In [ ]:
model_path = 'model.pt'
model = torch.load(model_path)
model.eval()

In [ ]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, seed=None):

  indices = vocab(tokenizer(prompt))
  itos = vocab.get_itos()

  for i in range(max_seq_len):
    src = torch.LongTensor(indices).to(device)
    with torch.no_grad():
      prediction = model(src)

    # Low values like 0.1 for temperature, Makes softmax like argmax more
    probs = torch.softmax(prediction[-1]/temperature, dim = 0)
    idx = vocab["<ukn>"]
    while idx == vocab["<ukn>"]:
      idx = torch.multinomial(probs, num_samples =1).item()
    indices.append(idx)
    prompt += " " + itos[idx]
    # print(prompt)

    if idx == vocab["."]:
      return prompt

In [ ]:
prompt = "as i know about this subject,"
generate(prompt, 40, 0.5, model, tokenizer, vocab, seed=None)